Author: Matthew Forbes
Date: May 12th, 2022
ZEDZ Shipment Model for APP

The following code produces estimates for the total driving distance conducted by delivery vehicles in the City of Santa Monica. It also produces estimates on how these driving distances change when different ZEDZ policies are imposed, using the boundaries of the voluntary-ZEDZ that is currently being implemented. 

This model is based on "Application of an empirical multi-agent model for urban goods transport to analyze impacts of zero emission zones in The Netherlands" by Michiel de Bok, Lorant Tavasszy, and Sebastiaan Thoen: https://www.sciencedirect.com/science/article/pii/S0967070X19307383. The model contains three main parts: shipment synthesizer, tour formation, and network assignment.

The first part, shipment synthesizer, is tasked with generating a dataframe of possible shipments that could plausibly occur on an average day. This dataframe contains the following for each generated shipment:
● The name and location of the sender;
● The name and location of the receiver;
● The company that is conducting this shipment

The second section, tour formation, is concerned with estimating the number of shipments contained in a single shipment route and then grouping these shipments together (i.e. can multiple shipments be completed in a single tour of a shipment vehicle, and if so which shipments are likely to be grouped together).

The third section, network assignment, attempts to predict the potential routes that vehicles will take to complete these tours. These estimates are derived simply by mapping the shortest path from the sender location that connects all recipient locations.

Inputs (these need to be in your working directory)(author: Ryota Abe):
- "consumer_nodes edited.csv"
- "depot_nodes edited.csv"
- "Entire Nodes.xlsx"
- "consumer_nodes_new.csv"
- "depot-x_arccs.xlsx" with x being 1 through 16

Outputs:
- "FinalResults - status quo dist.xlsx"
    -the total distance driven in meters by each company and in total under no policy
- "FinalResults - mandatory ZEDZ dist.xlsx"
    -the total distance driven in meters by each company and in total under a mandatory ZEDZ policy
- "FinalResults - status quo tours.xlsx"
    -the number of vehicle tours conducted by each company and in total under no policy
- "FinalResults - mandatory ZEDZ tours.xlsx"
    -the number of vehicle tours conducted by each company and in total under a mandatory ZEDZ policy
- "FinalResults - status quo vehicles.xlsx"
    -the distance driven by tour and vehicle under no policy
- "FinalResults - mandatory ZEDZ vehicles.xlsx"
    -the distance driven by tour and vehicle under a mandatory ZEDZ policy

*Model description taken from our APP report

In [1]:
#Import Statements
import pandas as pd 
import numpy as np
import random

In [2]:
#Import consumer nodes
consumer_nodes = pd.read_csv("consumer_nodes edited.csv")
consumer_nodes

,street_id,street_address,latitude,longitude,ZEDZ(inside =1),pop_distribution,parcel_delivered(#/year),parcel_delivered(#/day),parcel_delivered(kg/day),USPS_Depot_name,...,UPS_Depot_name,UPS_Depot_parcel(kg),Amazon_Depot_name,Amazon_Depot_Depot_parcel(kg),FedEX_Depot_name,FedEX_Depot_parcel(kg),USPS_ID,UPS_ID,Amazon_ID,FedEx_ID
0,1,"10th Court, Santa Monica, CA90403",34.019890,-118.488409,0,263.125356,7367.509972,20.184959,22.203455,depot-5,...,depot-9,5.328829,depot-10,4.662725,depot-12,3.996622,5,9,10,12
1,2,"10th Court, Santa Monica, CA90405",34.019897,-118.487261,0,263.125356,7367.509972,20.184959,22.203455,depot-5,...,depot-9,5.328829,depot-10,4.662725,depot-12,3.996622,5,9,10,12
2,3,"10th Street, Santa Monica, CA90401",34.019890,-118.488409,0,263.125356,7367.509972,20.184959,22.203455,depot-5,...,depot-9,5.328829,depot-10,4.662725,depot-12,3.996622,5,9,10,12
3,4,"10th Street, Santa Monica, CA90402",34.019890,-118.488409,0,263.125356,7367.509972,20.184959,22.203455,depot-5,...,depot-9,5.328829,depot-10,4.662725,depot-12,3.996622,5,9,10,12
4,5,"10th Street, Santa Monica, CA90403",34.019890,-118.488409,0,263.125356,7367.509972,20.184959,22.203455,depot-5,...,depot-9,5.328829,depot-10,4.662725,depot-12,3.996622,5,9,10,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,347,"Winnett Place, Santa Monica, CA90402",34.031853,-118.502660,0,263.125356,7367.509972,20.184959,22.203455,depot-1,...,depot-9,5.328829,depot-10,4.662725,depot-12,3.996622,1,9,10,12
347,348,"Woodacres Road, Santa Monica, CA90402",34.039950,-118.505475,0,263.125356,7367.509972,20.184959,22.203455,depot-1,...,depot-9,5.328829,depot-10,4.662725,depot-12,3.996622,1,9,10,12
348,349,"Yale Street, Santa Monica, CA90403",34.038366,-118.475060,0,263.125356,7367.509972,20.184959,22.203455,depot-2,...,depot-9,5.328829,depot-10,4.662725,depot-13,3.996622,2,9,10,13
349,350,"Yale Street, Santa Monica, CA90404",34.034314,-118.470567,0,263.125356,7367.509972,20.184959,22.203455,depot-3,...,depot-9,5.328829,depot-10,4.662725,depot-14,3.996622,3,9,10,14


In [3]:
#import logistic nodes
logistic_nodes = pd.read_csv("depot_nodes edited.csv")
logistic_nodes

,logistics_company_id,logistics_company_name,logistics_company_share,depot_id,depot_share,depot_parcel(#),depot_parcel(kg),depot_address,depot_latitude,depot_longitude,ZEDZ(inside =1)
0,1,USPS,0.38,1,0.130772,935.774691,1029.352160,"1112 Montana Ave, Santa Monica, CA 90403",34.030044,-118.497416,0
1,1,USPS,0.38,2,0.012863,92.043412,101.247754,"1626 Montana Ave, Santa Monica, CA 90403",34.034159,-118.492490,0
2,1,USPS,0.38,3,0.052523,92.043412,413.428327,"1223 Wilshire Blvd, Santa Monica, CA 90403",34.025935,-118.490306,0
3,1,USPS,0.38,4,0.005360,92.043412,42.186564,"1217 Wilshire Blvd, Santa Monica, CA 90403",34.025861,-118.490424,0
4,1,USPS,0.38,5,0.056811,92.043412,447.177578,"1653 7th St, Santa Monica, CA 90404",34.015040,-118.487514,1
5,1,USPS,0.38,6,0.099687,92.043412,784.670089,"2720 Neilson Way Fl 1, Santa Monica, CA 90405",33.999954,-118.482961,1
6,1,USPS,0.38,7,0.003216,92.043412,25.311938,"313 Grand Blvd, Venice, CA 90291",33.988641,-118.469427,0
7,1,USPS,0.38,8,0.015007,92.043412,118.122379,"3826 Grand View Blvd, Los Angeles, CA 90066",34.004446,-118.430180,0
8,2,UPS,0.24,9,0.237624,92.043412,1870.419025,"10690 Santa Monica Blvd, Los Angeles, CA 90025",34.051950,-118.431388,0
9,3,Amazon Logistics,0.21,10,0.207921,92.043412,1636.616647,"5750 Mesmer Ave STE D, Culver City, CA 90230",33.984066,-118.398651,0


In [4]:
#Import depot arcs
depot_1 = pd.read_excel("depot-1_arccs.xlsx")
depot_2 = pd.read_excel("depot-2_arcs.xlsx")
depot_3 = pd.read_excel("depot-3_arcs.xlsx")
depot_4 = pd.read_excel("depot-4_arccs.xlsx")
depot_5 = pd.read_excel("depot-5_arccs.xlsx")
depot_6 = pd.read_excel("depot-6_arccs.xlsx")
depot_7 = pd.read_excel("depot-7_arccs.xlsx")
depot_8 = pd.read_excel("depot-8_arccs.xlsx")
depot_9 = pd.read_excel("depot-9_arccs.xlsx")
depot_10 = pd.read_excel("depot-10_arccs.xlsx")
depot_11 = pd.read_excel("depot-11_arccs.xlsx")
depot_12 = pd.read_excel("depot-12_arccs.xlsx")
depot_13 = pd.read_excel("depot-13_arccs.xlsx")
depot_14 = pd.read_excel("depot-14_arccs.xlsx")
depot_15 = pd.read_excel("depot-15_arccs.xlsx")
depot_16 = pd.read_excel("depot-16_arccs.xlsx")

In [5]:
#Merge depot dataframes together
depot_arcs = pd.concat([depot_1, depot_2, depot_3, depot_4, depot_5, depot_6, depot_7, depot_8, depot_9, depot_10, depot_11, depot_12, depot_13, depot_14, depot_15, depot_16])
depot_arcs

,Unnamed: 0,Distance in meter,duration in seconds,origin_id,destination_id,ID_x,longitude_x,latitude_x,location_x,ID_y,longitude_y,latitude_y,location_y,spot_x,spot_y,city_x,city_y
0,0,0,0,1,1,1,-118.497420,34.030040,Depot-1,1,-118.497420,34.030040,Depot-1,NaN,NaN,NaN,NaN
1,1,24,85,1,24,1,-118.497420,34.030040,Depot-1,24,-118.497364,34.029789,Street-8,NaN,NaN,NaN,NaN
2,2,1339,231,1,91,1,-118.497420,34.030040,Depot-1,91,-118.506132,34.022201,Street-75,NaN,NaN,NaN,NaN
3,3,1339,231,1,93,1,-118.497420,34.030040,Depot-1,93,-118.506132,34.022201,Street-77,NaN,NaN,NaN,NaN
4,4,1380,255,1,106,1,-118.497420,34.030040,Depot-1,106,-118.500623,34.021548,Street-90,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,26921,1581,313,299,313,-118.449599,34.014784,NaN,299,-118.247567,34.086434,NaN,NaN,NaN,NaN,NaN
97,97,4030,492,313,310,313,-118.449599,34.014784,NaN,310,-118.454339,33.995917,NaN,NaN,NaN,NaN,NaN
98,98,1643,212,313,312,313,-118.449599,34.014784,NaN,312,-118.463517,34.006780,NaN,NaN,NaN,NaN,NaN
99,99,0,0,313,313,313,-118.449599,34.014784,NaN,313,-118.449599,34.014784,NaN,NaN,NaN,NaN,NaN


In [6]:
#Import Entire Nodes dataset
entire_nodes = pd.read_excel("Entire Nodes.xlsx")
entire_nodes

,ID,depot_id,street_id,longitude,latitude,location,USPS Depot,pd(kg),UPS Depot,pd(kg).1,Amazon Depot,pd(kg).2,FedEX Depot,pd(kg).3
0,1,1,-,-118.497420,34.030040,Depot-1,Depot-1,-,-,-,-,-,-,-
1,2,2,-,-118.492490,34.034160,Depot-2,Depot-2,-,-,-,-,-,-,-
2,3,3,-,-118.490310,34.025930,Depot-3,Depot-3,-,-,-,-,-,-,-
3,4,4,-,-118.490420,34.025860,Depot-4,Depot-4,-,-,-,-,-,-,-
4,5,5,-,-118.487510,34.015040,Depot-5,Depot-5,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,363,-,347,-118.502660,34.031853,Street-347,Depot-1,8.437313,Depot-9,5.328829,Depot-10,4.662725,Depot-12,3.996622
363,364,-,348,-118.505475,34.039950,Street-348,Depot-1,8.437313,Depot-9,5.328829,Depot-10,4.662725,Depot-12,3.996622
364,365,-,349,-118.475060,34.038366,Street-349,Depot-2,8.437313,Depot-9,5.328829,Depot-10,4.662725,Depot-13,3.996622
365,366,-,350,-118.470567,34.034314,Street-350,Depot-3,8.437313,Depot-9,5.328829,Depot-10,4.662725,Depot-14,3.996622


<font size="5">Part I: Shipment Synthesizer</font>

In [1]:
#Part 1.1: Depot Proportions
#This part declares the proportions of shipments that originate from each logistic depot

depots = logistic_nodes["depot_id"]
depot_share = logistic_nodes["depot_share"]

#Share of shipments that originate from each company (data from APP report)
company_share = [0.38, 0.24, 0.21, 0.17]
company_names = ["USPS", "UPS", "Amazon Logistics", "FedEx"]

parcels_per_day = pd.read_csv("consumer_nodes_new.csv")
parcels_per_day = parcels_per_day["parcel_delivered(#/day)"].tolist()
parcels_per_day = [round(num) for num in parcels_per_day]

NameError: name 'logistic_nodes' is not defined

In [9]:
#Part 1.2: Generate Shipments

#The list of consumer node IDs
consumer_ids = consumer_nodes["street_id"]

#Dataframe that will ultimately contain all shipment data
shipment_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])

#Inputs a consumer node, and outputs the appropriate depot based on the company share proportions and the nearest depots to the consumer node.
def which_sender(consumer_id):
    the_consumer = consumer_nodes.iloc[consumer_id]
    the_business = np.random.choice( company_names, 1, p = company_share)
    if the_business[0] == "USPS":
        return the_consumer["USPS_ID"]
    if the_business[0] == "UPS":
        return the_consumer["UPS_ID"]
    if the_business[0] == "Amazon Logistics":
        return the_consumer["Amazon_ID"]
    if the_business[0] == "FedEx":
        return the_consumer["FedEx_ID"]
    
#Copy of consumer_node Dataframe
num_shipments = consumer_nodes

#Combines consumer and depot node information into the shipment_df dataframe, and selects appropriate sender depot for shipment to each node.
for i in range(len(consumer_ids)):
    ship_per_node =  parcels_per_day[i]
    for j in range(ship_per_node):
        receiver = consumer_nodes.iloc[i]
        sender_id = which_sender(i)
        sender = logistic_nodes.loc[logistic_nodes["depot_id"] == sender_id]

        receiver_id = receiver["street_id"]
        receiver_lat = receiver["latitude"]
        receiver_lon = receiver["longitude"]
        receiver_name = receiver["street_address"]
        rec_ZEDZ = receiver["ZEDZ(inside =1)"]

        sender_name = sender.iloc[0]["logistics_company_name"]
        sender_lat = sender.iloc[0]["depot_latitude"]
        sender_lon = sender.iloc[0]["depot_longitude"]
        sen_ZEDZ = sender.iloc[0]["ZEDZ(inside =1)"]
    
        shipment_df.loc[len(shipment_df.index)] = [receiver_name, receiver_id, receiver_lat, receiver_lon, rec_ZEDZ, sender_name, sender_id, sender_lat, sender_lon, sen_ZEDZ]
    




#Calculates the distance between each consumer node and its depot
ship_dist = []
for i in range(shipment_df[shipment_df.columns[0]].count()):
    the_shipment = shipment_df.iloc[[i]]
    the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
    the_depot_id = the_shipment.iloc[0]["Sender_ID"]
    #Get the Entire_nodes ID for the receiver
    rec_node = entire_nodes[entire_nodes['street_id'] == the_ship_id]
    receiver_entire_ID = rec_node.iloc[0]["ID"]
    #Get the Entire_nodes ID for the sender
    sen_node = entire_nodes[entire_nodes["depot_id"] == the_depot_id ]
    sender_entire_ID = sen_node.iloc[0]["ID"]

    
    poss_depo_arcs = depot_arcs[depot_arcs['origin_id'] == sender_entire_ID]
    the_arc = poss_depo_arcs[poss_depo_arcs['destination_id'] == receiver_entire_ID]
    the_arc_dist = the_arc.iloc[0]['Distance in meter']
    
    ship_dist.append(the_arc_dist)

#Adds distances to shipment_df
shipment_df["Distance"] = ship_dist
 
shipment_df



the shipID
1
the depotID
9
rec entire:
17
sen entire:
9
count
0
the shipID
1
the depotID
12
rec entire:
17
sen entire:
12
count
1
the shipID
1
the depotID
12
rec entire:
17
sen entire:
12
count
2
the shipID
1
the depotID
5
rec entire:
17
sen entire:
5
count
3
the shipID
1
the depotID
5
rec entire:
17
sen entire:
5
count
4
the shipID
1
the depotID
5
rec entire:
17
sen entire:
5
count
5
the shipID
1
the depotID
10
rec entire:
17
sen entire:
10
count
6
the shipID
1
the depotID
9
rec entire:
17
sen entire:
9
count
7
the shipID
1
the depotID
9
rec entire:
17
sen entire:
9
count
8
the shipID
1
the depotID
5
rec entire:
17
sen entire:
5
count
9
the shipID
1
the depotID
5
rec entire:
17
sen entire:
5
count
10
the shipID
1
the depotID
5
rec entire:
17
sen entire:
5
count
11
the shipID
2
the depotID
9
rec entire:
18
sen entire:
9
count
12
the shipID
2
the depotID
9
rec entire:
18
sen entire:
9
count
13
the shipID
2
the depotID
10
rec entire:
18
sen entire:
10
count
14
the shipID
2
the depotID
12

the shipID
12
the depotID
9
rec entire:
28
sen entire:
9
count
196
the shipID
12
the depotID
9
rec entire:
28
sen entire:
9
count
197
the shipID
12
the depotID
9
rec entire:
28
sen entire:
9
count
198
the shipID
12
the depotID
12
rec entire:
28
sen entire:
12
count
199
the shipID
12
the depotID
9
rec entire:
28
sen entire:
9
count
200
the shipID
12
the depotID
5
rec entire:
28
sen entire:
5
count
201
the shipID
12
the depotID
5
rec entire:
28
sen entire:
5
count
202
the shipID
12
the depotID
9
rec entire:
28
sen entire:
9
count
203
the shipID
12
the depotID
5
rec entire:
28
sen entire:
5
count
204
the shipID
12
the depotID
10
rec entire:
28
sen entire:
10
count
205
the shipID
12
the depotID
9
rec entire:
28
sen entire:
9
count
206
the shipID
12
the depotID
5
rec entire:
28
sen entire:
5
count
207
the shipID
12
the depotID
5
rec entire:
28
sen entire:
5
count
208
the shipID
12
the depotID
5
rec entire:
28
sen entire:
5
count
209
the shipID
12
the depotID
5
rec entire:
28
sen entire:
5
c

the shipID
18
the depotID
9
rec entire:
34
sen entire:
9
count
413
the shipID
19
the depotID
9
rec entire:
35
sen entire:
9
count
414
the shipID
19
the depotID
10
rec entire:
35
sen entire:
10
count
415
the shipID
19
the depotID
9
rec entire:
35
sen entire:
9
count
416
the shipID
19
the depotID
3
rec entire:
35
sen entire:
3
count
417
the shipID
19
the depotID
9
rec entire:
35
sen entire:
9
count
418
the shipID
19
the depotID
12
rec entire:
35
sen entire:
12
count
419
the shipID
19
the depotID
12
rec entire:
35
sen entire:
12
count
420
the shipID
19
the depotID
9
rec entire:
35
sen entire:
9
count
421
the shipID
19
the depotID
12
rec entire:
35
sen entire:
12
count
422
the shipID
19
the depotID
3
rec entire:
35
sen entire:
3
count
423
the shipID
19
the depotID
10
rec entire:
35
sen entire:
10
count
424
the shipID
19
the depotID
3
rec entire:
35
sen entire:
3
count
425
the shipID
20
the depotID
10
rec entire:
36
sen entire:
10
count
426
the shipID
20
the depotID
9
rec entire:
36
sen ent

the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
count
629
the shipID
29
the depotID
9
rec entire:
45
sen entire:
9
count
630
the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
count
631
the shipID
29
the depotID
9
rec entire:
45
sen entire:
9
count
632
the shipID
29
the depotID
14
rec entire:
45
sen entire:
14
count
633
the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
count
634
the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
count
635
the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
count
636
the shipID
29
the depotID
9
rec entire:
45
sen entire:
9
count
637
the shipID
29
the depotID
10
rec entire:
45
sen entire:
10
count
638
the shipID
29
the depotID
9
rec entire:
45
sen entire:
9
count
639
the shipID
29
the depotID
9
rec entire:
45
sen entire:
9
count
640
the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
count
641
the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
count
642
the shipID
29
the depotID
5
rec entire:
45
sen entire:
5
c

the shipID
36
the depotID
10
rec entire:
52
sen entire:
10
count
844
the shipID
36
the depotID
3
rec entire:
52
sen entire:
3
count
845
the shipID
36
the depotID
3
rec entire:
52
sen entire:
3
count
846
the shipID
36
the depotID
3
rec entire:
52
sen entire:
3
count
847
the shipID
36
the depotID
13
rec entire:
52
sen entire:
13
count
848
the shipID
36
the depotID
10
rec entire:
52
sen entire:
10
count
849
the shipID
36
the depotID
10
rec entire:
52
sen entire:
10
count
850
the shipID
36
the depotID
10
rec entire:
52
sen entire:
10
count
851
the shipID
36
the depotID
10
rec entire:
52
sen entire:
10
count
852
the shipID
36
the depotID
13
rec entire:
52
sen entire:
13
count
853
the shipID
36
the depotID
13
rec entire:
52
sen entire:
13
count
854
the shipID
36
the depotID
3
rec entire:
52
sen entire:
3
count
855
the shipID
36
the depotID
9
rec entire:
52
sen entire:
9
count
856
the shipID
36
the depotID
3
rec entire:
52
sen entire:
3
count
857
the shipID
36
the depotID
10
rec entire:
52
se

...

the shipID
300
the depotID
9
rec entire:
316
sen entire:
9
count
6246
the shipID
300
the depotID
9
rec entire:
316
sen entire:
9
count
6247
the shipID
300
the depotID
8
rec entire:
316
sen entire:
8
count
6248
the shipID
300
the depotID
8
rec entire:
316
sen entire:
8
count
6249
the shipID
300
the depotID
8
rec entire:
316
sen entire:
8
count
6250
the shipID
300
the depotID
8
rec entire:
316
sen entire:
8
count
6251
the shipID
300
the depotID
9
rec entire:
316
sen entire:
9
count
6252
the shipID
300
the depotID
8
rec entire:
316
sen entire:
8
count
6253
the shipID
300
the depotID
8
rec entire:
316
sen entire:
8
count
6254
the shipID
300
the depotID
14
rec entire:
316
sen entire:
14
count
6255
the shipID
300
the depotID
10
rec entire:
316
sen entire:
10
count
6256
the shipID
300
the depotID
9
rec entire:
316
sen entire:
9
count
6257
the shipID
300
the depotID
14
rec entire:
316
sen entire:
14
count
6258
the shipID
300
the depotID
8
rec entire:
316
sen entire:
8
count
6259
the shipID
300

the shipID
315
the depotID
12
rec entire:
331
sen entire:
12
count
6463
the shipID
315
the depotID
5
rec entire:
331
sen entire:
5
count
6464
the shipID
315
the depotID
5
rec entire:
331
sen entire:
5
count
6465
the shipID
315
the depotID
9
rec entire:
331
sen entire:
9
count
6466
the shipID
315
the depotID
9
rec entire:
331
sen entire:
9
count
6467
the shipID
315
the depotID
9
rec entire:
331
sen entire:
9
count
6468
the shipID
315
the depotID
5
rec entire:
331
sen entire:
5
count
6469
the shipID
315
the depotID
10
rec entire:
331
sen entire:
10
count
6470
the shipID
315
the depotID
10
rec entire:
331
sen entire:
10
count
6471
the shipID
315
the depotID
12
rec entire:
331
sen entire:
12
count
6472
the shipID
316
the depotID
12
rec entire:
332
sen entire:
12
count
6473
the shipID
316
the depotID
10
rec entire:
332
sen entire:
10
count
6474
the shipID
316
the depotID
12
rec entire:
332
sen entire:
12
count
6475
the shipID
316
the depotID
9
rec entire:
332
sen entire:
9
count
6476
the sh

the shipID
332
the depotID
5
rec entire:
348
sen entire:
5
count
6679
the shipID
333
the depotID
1
rec entire:
349
sen entire:
1
count
6680
the shipID
333
the depotID
10
rec entire:
349
sen entire:
10
count
6681
the shipID
333
the depotID
1
rec entire:
349
sen entire:
1
count
6682
the shipID
333
the depotID
9
rec entire:
349
sen entire:
9
count
6683
the shipID
333
the depotID
1
rec entire:
349
sen entire:
1
count
6684
the shipID
333
the depotID
10
rec entire:
349
sen entire:
10
count
6685
the shipID
333
the depotID
1
rec entire:
349
sen entire:
1
count
6686
the shipID
333
the depotID
10
rec entire:
349
sen entire:
10
count
6687
the shipID
333
the depotID
9
rec entire:
349
sen entire:
9
count
6688
the shipID
333
the depotID
11
rec entire:
349
sen entire:
11
count
6689
the shipID
333
the depotID
9
rec entire:
349
sen entire:
9
count
6690
the shipID
333
the depotID
11
rec entire:
349
sen entire:
11
count
6691
the shipID
333
the depotID
11
rec entire:
349
sen entire:
11
count
6692
the ship

the shipID
342
the depotID
2
rec entire:
358
sen entire:
2
count
6894
the shipID
342
the depotID
9
rec entire:
358
sen entire:
9
count
6895
the shipID
342
the depotID
13
rec entire:
358
sen entire:
13
count
6896
the shipID
342
the depotID
2
rec entire:
358
sen entire:
2
count
6897
the shipID
342
the depotID
9
rec entire:
358
sen entire:
9
count
6898
the shipID
343
the depotID
7
rec entire:
359
sen entire:
7
count
6899
the shipID
343
the depotID
9
rec entire:
359
sen entire:
9
count
6900
the shipID
343
the depotID
9
rec entire:
359
sen entire:
9
count
6901
the shipID
343
the depotID
10
rec entire:
359
sen entire:
10
count
6902
the shipID
343
the depotID
10
rec entire:
359
sen entire:
10
count
6903
the shipID
343
the depotID
7
rec entire:
359
sen entire:
7
count
6904
the shipID
343
the depotID
9
rec entire:
359
sen entire:
9
count
6905
the shipID
343
the depotID
10
rec entire:
359
sen entire:
10
count
6906
the shipID
343
the depotID
7
rec entire:
359
sen entire:
7
count
6907
the shipID
3

,Receiver,Receiver_ID,Receiver Lat,Receiver Lon,Receiver_ZEDZ,Sender,Sender_ID,Sender Lat,Sender Lon,Sender_ZEDZ,Distance
0,"10th Court, Santa Monica, CA90403",1,34.019890,-118.488409,0,UPS,9,34.051950,-118.431388,0,9448
1,"10th Court, Santa Monica, CA90403",1,34.019890,-118.488409,0,FedEx,12,34.023789,-118.492240,0,783
2,"10th Court, Santa Monica, CA90403",1,34.019890,-118.488409,0,FedEx,12,34.023789,-118.492240,0,783
3,"10th Court, Santa Monica, CA90403",1,34.019890,-118.488409,0,USPS,5,34.015040,-118.487514,1,740
4,"10th Court, Santa Monica, CA90403",1,34.019890,-118.488409,0,USPS,5,34.015040,-118.487514,1,740
...,...,...,...,...,...,...,...,...,...,...,...
7099,"Yorkshire Avenue, Santa Monica, CA90404",351,34.026252,-118.461364,0,USPS,5,34.015040,-118.487514,1,4409
7100,"Yorkshire Avenue, Santa Monica, CA90404",351,34.026252,-118.461364,0,Amazon Logistics,10,33.984066,-118.398651,0,9574
7101,"Yorkshire Avenue, Santa Monica, CA90404",351,34.026252,-118.461364,0,UPS,9,34.051950,-118.431388,0,5229
7102,"Yorkshire Avenue, Santa Monica, CA90404",351,34.026252,-118.461364,0,Amazon Logistics,10,33.984066,-118.398651,0,9574


<font size="5">Part II: Tour Formation + Part III: Network Assignment</font>

The Tour Formation section groups shipments together into "tours", which are groups of shipments that are completed by a single vehicle before having to return to the depot. The basis of this section comes from this Thoen et al. 2020 paper, which is also used in the de Bok paper: https://reader.elsevier.com/reader/sd/pii/S1366554520306402?token=94FFE679DDC95AB4588FD05E915EF54B466EA50544AAD164F8C79AFFB3D97B92F41DC853C84CBE6E1BFC66821C332BA8&originRegion=us-east-1&originCreation=20220214151704

This grouping is conducted in such a way that the tour is already arranged in an optimal way, also satisfying Part III.

In [10]:
#Tours without ZEDZ

UPS_tours = []
Amazon_tours = []
USPS_tours = []
FedEx_tours = []
#Our analysis found that diesel vehicles can carry 30 shipments before having to return to the depot
stops_per_vehicle = 30
max_dist = 20000000000

UPS_shipments = shipment_df.loc[shipment_df["Sender"] == "UPS"]
Amazon_shipments = shipment_df.loc[shipment_df["Sender"] == "Amazon Logistics"]
USPS_shipments = shipment_df.loc[shipment_df["Sender"] == "USPS"]
FedEx_shipments = shipment_df.loc[shipment_df["Sender"] == "FedEx"]

#Calculates the Euclidian distance between two locations
def calc_dist(A_lat, A_lon, B_lat, B_lon):
    dist = ((((A_lon - B_lon )**2) + ((A_lat - B_lat)**2) )**0.5)
    return dist

#Returns the actual driving distance between two locations
def calc_dist_nodes(A_ID, B_ID):
    #convert A_ID to its entire_node_ID
    A_node = entire_nodes[entire_nodes['street_id'] == A_ID]
    A_entire_ID = A_node.iloc[0]["ID"]
    
    #convert B_ID to its entire_node_ID
    B_node = entire_nodes[entire_nodes['street_id'] == B_ID]
    B_entire_ID = B_node.iloc[0]["ID"]
    
    poss_depo_arcs = depot_arcs[depot_arcs['origin_id'] == A_entire_ID]
    the_arc = poss_depo_arcs[poss_depo_arcs['destination_id'] == B_entire_ID]
    the_arc_dist = the_arc.iloc[0]['Distance in meter']
    return the_arc_dist

#The following functions generate a list of dataframes for each company.
#Each dataframe in these lists represent a single tour.
def UPS_tour_formation():
    remaining_shipments = pd.DataFrame.copy(UPS_shipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                
        else:
            UPS_tours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            if len(remaining_shipments.index) > 0:
                the_shipment = remaining_shipments.iloc[[0]]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    if len(tour_df.index) > 0:
        UPS_tours.append(tour_df)
        
def Amazon_tour_formation():
    remaining_shipments = pd.DataFrame.copy(Amazon_shipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            Amazon_tours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    Amazon_tours.append(tour_df)
    
def USPS_tour_formation():
    remaining_shipments = pd.DataFrame.copy(USPS_shipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            USPS_tours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    USPS_tours.append(tour_df)
    
def FedEx_tour_formation():
    remaining_shipments = pd.DataFrame.copy(FedEx_shipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            FedEx_tours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    FedEx_tours.append(tour_df)
    


UPS_tour_formation()
Amazon_tour_formation()
USPS_tour_formation()
FedEx_tour_formation()




In [11]:
#Tours with ZEDZ policy
UPS_Ztours = []
Amazon_Ztours = []
USPS_Ztours = []
FedEx_Ztours = []
UPS_nZtours = []
Amazon_nZtours = []
USPS_nZtours = []
FedEx_nZtours = []
max_dist = 200

UPS_Zshipments = shipment_df.loc[shipment_df["Sender"] == "UPS"]
UPS_Zshipments =  UPS_Zshipments[(UPS_Zshipments["Receiver_ZEDZ"] == 1) | (UPS_Zshipments["Sender_ZEDZ"] == 1)]
Amazon_Zshipments = shipment_df.loc[shipment_df["Sender"] == "Amazon Logistics"]
Amazon_Zshipments = Amazon_Zshipments[(Amazon_Zshipments["Receiver_ZEDZ"] == 1) | (Amazon_Zshipments["Sender_ZEDZ"] == 1)]
USPS_Zshipments = shipment_df.loc[shipment_df["Sender"] == "USPS"]
USPS_Zshipments = USPS_Zshipments[(USPS_Zshipments["Receiver_ZEDZ"] == 1) | (USPS_Zshipments["Sender_ZEDZ"] == 1)]
FedEx_Zshipments = shipment_df.loc[shipment_df["Sender"] == "FedEx"]
FedEx_Zshipments = FedEx_Zshipments[(FedEx_Zshipments["Receiver_ZEDZ"] == 1) | (FedEx_Zshipments["Sender_ZEDZ"] == 1)]

UPS_nZshipments = shipment_df.loc[shipment_df["Sender"] == "UPS"]
UPS_nZshipments =  UPS_nZshipments[(UPS_nZshipments["Receiver_ZEDZ"] == 0)]
UPS_nZshipments = UPS_nZshipments[(UPS_nZshipments["Sender_ZEDZ"] == 0)]
Amazon_nZshipments = shipment_df.loc[shipment_df["Sender"] == "Amazon Logistics"]
Amazon_nZshipments = Amazon_nZshipments[(Amazon_nZshipments["Receiver_ZEDZ"] == 0)]
Amazon_nZshipments = Amazon_nZshipments[(Amazon_nZshipments["Sender_ZEDZ"] == 0)]
USPS_nZshipments = shipment_df.loc[shipment_df["Sender"] == "USPS"]
USPS_nZshipments = USPS_nZshipments[(USPS_nZshipments["Receiver_ZEDZ"] == 0)]
USPS_nZshipments = USPS_nZshipments[(USPS_nZshipments["Sender_ZEDZ"] == 0)]
FedEx_nZshipments = shipment_df.loc[shipment_df["Sender"] == "FedEx"]
FedEx_nZshipments = FedEx_nZshipments[(FedEx_nZshipments["Receiver_ZEDZ"] == 0)]
FedEx_nZshipments = FedEx_nZshipments[(FedEx_nZshipments["Sender_ZEDZ"] == 0)]

tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
new_tour_df = pd.DataFrame.copy(tour_df)

#EV vehicles can do 33 stops from our analysis (mainly because they don't have to refuel as often)
stops_per_vehicle = 33


#The following functions generate a list of dataframes for each company and for whether the vehicle enters the ZEDZ zone.
#Each dataframe in these lists represent a single tour.
#Functions with "Z" are EVs that must enter the zone, and "nZ" do not
def UPS_Ztour_formation():
    remaining_shipments = pd.DataFrame.copy(UPS_Zshipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            UPS_Ztours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    UPS_Ztours.append(tour_df)
    
#This variable is redefined since the vehicles are now diesel
stops_per_vehicle = 30

def UPS_nZtour_formation():
    remaining_shipments = pd.DataFrame.copy(UPS_nZshipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            UPS_nZtours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    UPS_nZtours.append(tour_df)
   
stops_per_vehicle = 33

def Amazon_Ztour_formation():
    remaining_shipments = pd.DataFrame.copy(Amazon_Zshipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            Amazon_Ztours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    Amazon_Ztours.append(tour_df)
   
stops_per_vehicle = 30

def Amazon_nZtour_formation():
    remaining_shipments = pd.DataFrame.copy(Amazon_nZshipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            Amazon_nZtours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    Amazon_nZtours.append(tour_df)
    
stops_per_vehicle = 33

def USPS_Ztour_formation():
    remaining_shipments = pd.DataFrame.copy(USPS_Zshipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            USPS_Ztours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    USPS_Ztours.append(tour_df)
    
stops_per_vehicle = 30

def USPS_nZtour_formation():
    remaining_shipments = pd.DataFrame.copy(USPS_nZshipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    new_tour_df = pd.DataFrame.copy(tour_df)
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            USPS_nZtours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    USPS_nZtours.append(tour_df)
    
stops_per_vehicle = 33    

def FedEx_Ztour_formation():
    remaining_shipments = pd.DataFrame.copy(FedEx_Zshipments)
    tour_df = pd.DataFrame(columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"] )
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if (len(tour_df)) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            FedEx_Ztours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    FedEx_Ztours.append(tour_df)
    
    
stops_per_vehicle = 30    
    
def FedEx_nZtour_formation():
    remaining_shipments = pd.DataFrame.copy(FedEx_nZshipments)
    tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
    for i in range(remaining_shipments[remaining_shipments.columns[0]].count()):
        if len(remaining_shipments.index) == 0:
            break
        if len(tour_df) < stops_per_vehicle:
            if len(tour_df) == 0:
                the_shipment = remaining_shipments.iloc[:1]
                the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
                tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
                tour_df.reset_index()
            else:
                starting_lat = tour_df.iloc[0]["Receiver Lat"]
                starting_lon = tour_df.iloc[0]["Receiver Lon"]
                starting_ID = tour_df.iloc[0]["Receiver_ID"]
                min_dist = 1000000
                for k in range(len(remaining_shipments)):
                    q = remaining_shipments.iloc[[k]]
                    k_lat = q.iloc[0]["Receiver Lat"]
                    k_lon = q.iloc[0]["Receiver Lon"]
                    k_ID = q.iloc[0]["Receiver_ID"]
                    the_dist = calc_dist_nodes(starting_ID, k_ID)
                    if the_dist < min_dist:
                        the_chosen = q
                        min_dist = the_dist
                tour_df = pd.concat([tour_df, the_chosen], ignore_index = True)
                the_ship_id = the_chosen.iloc[0]["Receiver_ID"]
                remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
        else:
            FedEx_nZtours.append(tour_df)
            tour_df = pd.DataFrame( columns = [ "Receiver", "Receiver_ID", "Receiver Lat", "Receiver Lon", "Receiver_ZEDZ", "Sender", "Sender_ID", "Sender Lat", "Sender Lon", "Sender_ZEDZ"])
            the_shipment = remaining_shipments.iloc[[0]]
            the_ship_id = the_shipment.iloc[0]["Receiver_ID"]
            remaining_shipments = remaining_shipments[remaining_shipments["Receiver_ID"] != the_ship_id]
            tour_df = pd.concat([tour_df, the_shipment], ignore_index = True)
    FedEx_nZtours.append(tour_df)
    

    
UPS_Ztour_formation()
UPS_nZtour_formation()
Amazon_Ztour_formation()
Amazon_nZtour_formation()
USPS_Ztour_formation()
USPS_nZtour_formation()
FedEx_Ztour_formation()
FedEx_nZtour_formation()


In [13]:
#Calculate Distances
#This section calculates the total vehicle miles driven under each scenario, company, and type of vehicle.

#Status Quo - No ZEDZ
UPS_dist = 0
for i in range(len(UPS_tours)):
    the_tour = UPS_tours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            UPS_dist = UPS_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            UPS_dist = UPS_dist + calc_dist_nodes(start_id, end_id)
Amazon_dist = 0
for i in range(len(Amazon_tours)):
    the_tour = Amazon_tours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            Amazon_dist = Amazon_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            Amazon_dist = Amazon_dist + calc_dist_nodes(start_id, end_id)
USPS_dist = 0
for i in range(len(USPS_tours)):
    the_tour = USPS_tours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            USPS_dist = USPS_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            USPS_dist = USPS_dist + calc_dist_nodes(start_id, end_id)
FedEx_dist = 0
for i in range(len(FedEx_tours)):
    the_tour = FedEx_tours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            FedEx_dist = FedEx_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            FedEx_dist = FedEx_dist + calc_dist_nodes(start_id, end_id) 
    
total_dist_Status_Quo = UPS_dist + Amazon_dist + USPS_dist + FedEx_dist
    
    
#Mandatory ZEDZ
#Diesel Meters
UPS_nZ_dist = 0
for i in range(len(UPS_nZtours)):
    the_tour = UPS_nZtours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            UPS_nZ_dist = UPS_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            UPS_nZ_dist = UPS_nZ_dist + calc_dist_nodes(start_id, end_id)
Amazon_nZ_dist = 0
for i in range(len(Amazon_nZtours)):
    the_tour = Amazon_nZtours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            Amazon_nZ_dist = Amazon_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            Amazon_nZ_dist = Amazon_nZ_dist + calc_dist_nodes(start_id, end_id)
USPS_nZ_dist = 0
for i in range(len(USPS_nZtours)):
    the_tour = USPS_nZtours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            USPS_nZ_dist = USPS_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            USPS_nZ_dist = USPS_nZ_dist + calc_dist_nodes(start_id, end_id)
FedEx_nZ_dist = 0
for i in range(len(FedEx_nZtours)):
    the_tour = FedEx_nZtours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            FedEx_nZ_dist = FedEx_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            FedEx_nZ_dist = FedEx_nZ_dist + calc_dist_nodes(start_id, end_id)
    
total_dist_Man_Dies = FedEx_nZ_dist + USPS_nZ_dist + Amazon_nZ_dist + UPS_nZ_dist

#EV Meters
UPS_Z_dist = 0
for i in range(len(UPS_Ztours)):
    the_tour = UPS_Ztours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            UPS_Z_dist = UPS_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            UPS_Z_dist = UPS_Z_dist + calc_dist_nodes(start_id, end_id)
Amazon_Z_dist = 0
for i in range(len(Amazon_Ztours)):
    the_tour = Amazon_Ztours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            Amazon_Z_dist = Amazon_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            Amazon_Z_dist = Amazon_Z_dist + calc_dist_nodes(start_id, end_id)
USPS_Z_dist = 0
for i in range(len(USPS_Ztours)):
    the_tour = USPS_Ztours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            USPS_Z_dist = USPS_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            USPS_Z_dist = USPS_Z_dist + calc_dist_nodes(start_id, end_id)
FedEx_Z_dist = 0
for i in range(len(FedEx_Ztours)):
    the_tour = FedEx_Ztours[i]
    for j in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            FedEx_Z_dist = FedEx_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[j-1]["Receiver_ID"]
            end_id = the_tour.iloc[j]["Receiver_ID"]
            FedEx_Z_dist = FedEx_Z_dist + calc_dist_nodes(start_id, end_id)
    
total_dist_Man_EV = UPS_Z_dist + Amazon_Z_dist + USPS_Z_dist + FedEx_Z_dist

total_dist_mandatory = total_dist_Man_Dies + total_dist_Man_EV

In [14]:
#Calculate total tours

#Status Quo - No ZEDZ
UPS_tours_count = len(UPS_tours)

Amazon_tours_count = len(Amazon_tours)

USPS_tours_count = len(USPS_tours)

FedEx_tours_count = len(FedEx_tours)
    
total_tours_Status_Quo = UPS_tours_count + Amazon_tours_count + USPS_tours_count + FedEx_tours_count

#Mandatory ZEDZ
#Diesel tours
UPS_nZ_tours_count = len(UPS_nZtours)

Amazon_nZ_tours_count = len(Amazon_nZtours)

USPS_nZ_tours_count = len(USPS_nZtours)

FedEx_nZ_tours_count = len(FedEx_nZtours)

total_tours_Mand_Dies = UPS_nZ_tours_count + Amazon_nZ_tours_count + USPS_nZ_tours_count + FedEx_nZ_tours_count

#EV tours
UPS_Z_tours_count = len(UPS_Ztours) 

Amazon_Z_tours_count = len(Amazon_Ztours)

USPS_Z_tours_count = len(USPS_Ztours)

FedEx_Z_tours_count = len(FedEx_Ztours)

total_tours_Mand_EV = UPS_Z_tours_count  + Amazon_Z_tours_count + USPS_Z_tours_count + FedEx_Z_tours_count

total_tours_Mand = total_tours_Mand_Dies + total_tours_Mand_EV


In [15]:
#Create dataframes of dist traveled

#status quo
status_quo = pd.DataFrame( columns = ["UPS_dist", "Amazon_dist", "USPS_dist", "FedEx_dist", "Total_dist" ])
status_quo["UPS_dist"] = [UPS_dist]
status_quo["Amazon_dist"] = [Amazon_dist]
status_quo["USPS_dist"] = [USPS_dist]
status_quo["FedEx_dist"] = [FedEx_dist]
status_quo["Total_dist"] = [total_dist_Status_Quo]


#mandatory ZEDZ
mandatory_ZEDZ = pd.DataFrame( columns = ["UPS_nZdist", "Amazon_nZdist", "USPS_nZdist", "FedEx_nZdist", "Total_nZdist", "UPS_Zdist", "Amazon_Zdist", "USPS_Zdist", "FedEx_Zdist", "Total_Zdist", "Total_dist" ])
mandatory_ZEDZ["UPS_nZdist"] = [UPS_nZ_dist]
mandatory_ZEDZ["Amazon_nZdist"] = [Amazon_nZ_dist]
mandatory_ZEDZ["USPS_nZdist"] = [USPS_nZ_dist]
mandatory_ZEDZ["FedEx_nZdist"] = [FedEx_nZ_dist]
mandatory_ZEDZ["Total_nZdist"] = [total_dist_Man_Dies]

mandatory_ZEDZ["UPS_Zdist"] = [UPS_Z_dist]
mandatory_ZEDZ["Amazon_Zdist"] = [Amazon_Z_dist]
mandatory_ZEDZ["USPS_Zdist"] = [USPS_Z_dist]
mandatory_ZEDZ["FedEx_Zdist"] = [FedEx_Z_dist]
mandatory_ZEDZ["Total_Zdist"] = [total_dist_Man_EV]

mandatory_ZEDZ["Total_dist"] = [total_dist_mandatory]

#Create dataframes of tours conducted
#status quo
status_quo_tours = pd.DataFrame( columns = ["UPS_tours", "Amazon_tours", "USPS_tours", "FedEx_tours", "Total_tours" ])
status_quo_tours["UPS_tours"] = [UPS_tours_count]
status_quo_tours["Amazon_tours"] = [Amazon_tours_count]
status_quo_tours["USPS_tours"] = [USPS_tours_count]
status_quo_tours["FedEx_tours"] = [FedEx_tours_count]
status_quo_tours["Total_tours"] = [total_tours_Status_Quo]

#mandatory ZEDZ
mandatory_ZEDZ_tours = pd.DataFrame( columns = ["UPS_nZ_tours", "Amazon_nZ_tours", "USPS_nZ_tours", "FedEx_nZ_tours", "Total_nZ_tours", "UPS_Z_tours", "Amazon_Z_tours", "USPS_Z_tours", "FedEx_Z_tours", "Total_Z_tours", "Total_tours" ])
mandatory_ZEDZ_tours["UPS_nZ_tours"] = [UPS_nZ_tours_count]
mandatory_ZEDZ_tours["Amazon_nZ_tours"] = [Amazon_nZ_tours_count]
mandatory_ZEDZ_tours["USPS_nZ_tours"] = [USPS_nZ_tours_count]
mandatory_ZEDZ_tours["FedEx_nZ_tours"] = [FedEx_nZ_tours_count]
mandatory_ZEDZ_tours["Total_nZ_tours"] = [total_tours_Mand_Dies]

mandatory_ZEDZ_tours["UPS_Z_tours"] = [UPS_Z_tours_count]
mandatory_ZEDZ_tours["Amazon_Z_tours"] = [Amazon_Z_tours_count]
mandatory_ZEDZ_tours["USPS_Z_tours"] = [USPS_Z_tours_count]
mandatory_ZEDZ_tours["FedEx_Z_tours"] = [FedEx_Z_tours_count]
mandatory_ZEDZ_tours["Total_Z_tours"] = [total_tours_Mand_EV]

mandatory_ZEDZ_tours["Total_tours"] = [total_tours_Mand]

In [17]:
status_quo.to_excel('FinalResults - status quo dist.xlsx')

In [18]:
mandatory_ZEDZ.to_excel('FinalResults - mandatory ZEDZ dist.xlsx')

In [19]:
status_quo_tours.to_excel('FinalResults - status quo tours.xlsx')

In [20]:
mandatory_ZEDZ_tours.to_excel('FinalResults - mandatory ZEDZ tours.xlsx')

In [21]:
#Generate distance per tour
#status quo
status_quo_vehicles = pd.DataFrame(columns = ["Company", "Vehicle", "Tour", "Distance by tour" ])
company = []
vehicle = []
tour = []
Distance_tour = []
Distance_vehicle = []
vehicle_number = 1
tour_num = 0
j = 0
for i in range(len(UPS_tours)):
    tour_num += 1
    tour.append(tour_num)
    company.append("UPS")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(UPS_tours) - 1:
            vehicle_number += 1
    the_tour = UPS_tours[i]
    UPS_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            UPS_dist = UPS_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            UPS_dist = UPS_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(UPS_dist)
j = 0
vehicle_number += 1
for i in range(len(Amazon_tours)):
    tour_num += 1
    tour.append(tour_num)
    company.append("Amazon")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(Amazon_tours) - 1:
            vehicle_number += 1
    the_tour = Amazon_tours[i]
    Amazon_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            Amazon_dist = Amazon_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            Amazon_dist = Amazon_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(Amazon_dist)
    
j = 0
vehicle_number += 1
for i in range(len(USPS_tours)):
    tour_num += 1
    tour.append(tour_num)
    company.append("USPS")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(USPS_tours) - 1:
            vehicle_number += 1
    the_tour = USPS_tours[i]
    USPS_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if j == 0: 
            USPS_dist = USPS_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            USPS_dist = USPS_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(USPS_dist)

j = 0
vehicle_number += 1
for i in range(len(FedEx_tours)):
    tour_num += 1
    tour.append(tour_num)
    company.append("FedEx")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(FedEx_tours) - 1:
            vehicle_number += 1
    the_tour = FedEx_tours[i]
    FedEx_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            FedEx_dist = FedEx_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            FedEx_dist = FedEx_dist + calc_dist_nodes(start_id, end_id) 
    Distance_tour.append(FedEx_dist)

status_quo_vehicles["Company"] = company
status_quo_vehicles["Vehicle"] = vehicle
status_quo_vehicles["Tour"] = tour
status_quo_vehicles["Distance by tour"] = Distance_tour

temp = status_quo_vehicles.groupby(["Vehicle"]).sum()
temp = temp.rename(columns={"Distance by tour": "Distance by vehicle"})
temp = temp.drop(["Tour"], axis = 1)

status_quo_vehicles = pd.merge(status_quo_vehicles, temp, on = "Vehicle")


#Mandatory ZEDZ
mandatory_vehicles = pd.DataFrame(columns = ["Company", "Vehicle", "Tour", "Distance by tour", "EV" ])
company = []
vehicle = []
tour = []
Distance_tour = []
Distance_vehicle = []
EV = []
vehicle_number = 1
tour_num = 0
j = 0
for i in range(len(UPS_nZtours)):
    tour_num += 1
    tour.append(tour_num)
    EV.append(0)
    company.append("UPS")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(UPS_nZtours) - 1:
            vehicle_number += 1
    the_tour = UPS_nZtours[i]
    UPS_nZ_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            UPS_nZ_dist = UPS_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            UPS_nZ_dist = UPS_nZ_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(UPS_nZ_dist)
j = 0
vehicle_number += 1
for i in range(len(Amazon_nZtours)):
    tour_num += 1
    tour.append(tour_num)
    EV.append(0)
    company.append("Amazon")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(Amazon_nZtours) - 1:
            vehicle_number += 1
    the_tour = Amazon_nZtours[i]
    Amazon_nZ_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            Amazon_nZ_dist = Amazon_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            Amazon_nZ_dist = Amazon_nZ_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(Amazon_nZ_dist)
    
j = 0
vehicle_number += 1
for i in range(len(USPS_nZtours)):
    tour_num += 1
    tour.append(tour_num)
    EV.append(0)
    company.append("USPS")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(USPS_nZtours) - 1:
            vehicle_number += 1
    the_tour = USPS_nZtours[i]
    USPS_nZ_dist = 0 
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            USPS_nZ_dist = USPS_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            USPS_nZ_dist = USPS_nZ_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(USPS_nZ_dist)
    
j = 0
vehicle_number += 1
for i in range(len(FedEx_nZtours)):
    tour_num += 1
    tour.append(tour_num)
    EV.append(0)
    company.append("FedEx")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(FedEx_nZtours) - 1:
            vehicle_number += 1
    the_tour = FedEx_nZtours[i]
    FedEx_nZ_dist = 0 
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            FedEx_nZ_dist = FedEx_nZ_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            FedEx_nZ_dist = FedEx_nZ_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(FedEx_nZ_dist)
    
j = 0
vehicle_number += 1
for i in range(len(UPS_Ztours)):
    tour_num += 1
    tour.append(tour_num)
    company.append("UPS")
    EV.append(1)
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(UPS_Ztours) - 1:
            vehicle_number += 1
    the_tour = UPS_Ztours[i]
    UPS_Z_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            UPS_Z_dist = UPS_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            UPS_Z_dist = UPS_Z_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(UPS_Z_dist)
    
j = 0
vehicle_number += 1
for i in range(len(Amazon_Ztours)):
    tour_num += 1
    tour.append(tour_num)
    EV.append(1)
    company.append("Amazon")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(Amazon_Ztours) - 1:
            vehicle_number += 1
    the_tour = Amazon_Ztours[i]
    Amazon_Z_dist = 0
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            Amazon_Z_dist = Amazon_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            Amazon_Z_dist = Amazon_Z_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(Amazon_Z_dist)

j = 0
vehicle_number += 1
for i in range(len(USPS_Ztours)):
    tour_num += 1
    tour.append(tour_num)
    company.append("USPS")
    EV.append(1)
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(USPS_Ztours) - 1:
            vehicle_number += 1
    the_tour = USPS_Ztours[i]
    USPS_Z_dist = 0 
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            USPS_Z_dist = USPS_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            USPS_Z_dist = USPS_Z_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(USPS_Z_dist)
    
j = 0
vehicle_number += 1
for i in range(len(FedEx_Ztours)):
    tour_num += 1
    tour.append(tour_num)
    EV.append(1)
    company.append("FedEx")
    if j == 0:
        vehicle.append(vehicle_number)
        j = 1
    elif j == 1:
        j = 0
        vehicle.append(vehicle_number)
        if i != len(FedEx_Ztours) - 1:
            vehicle_number += 1
    the_tour = FedEx_Ztours[i]
    FedEx_Z_dist = 0 
    for h in range(the_tour[the_tour.columns[0]].count()):
        if h == 0: 
            FedEx_Z_dist = FedEx_Z_dist + the_tour.iloc[0]["Distance"]
        else:
            start_id = the_tour.iloc[h-1]["Receiver_ID"]
            end_id = the_tour.iloc[h]["Receiver_ID"]
            FedEx_Z_dist = FedEx_Z_dist + calc_dist_nodes(start_id, end_id)
    Distance_tour.append(FedEx_Z_dist)
    
mandatory_vehicles["Company"] = company
mandatory_vehicles["Vehicle"] = vehicle
mandatory_vehicles["Tour"] = tour
mandatory_vehicles["Distance by tour"] = Distance_tour
mandatory_vehicles["EV"] = EV

temp = mandatory_vehicles.groupby(["Vehicle"]).sum()
temp = temp.rename(columns={"Distance by tour": "Distance by vehicle"})
temp = temp.drop(["Tour"], axis = 1)

mandatory_vehicles = pd.merge(mandatory_vehicles, temp, on = "Vehicle")

mandatory_vehicles = mandatory_vehicles.drop(["EV_y"], axis = 1)
mandatory_vehicles = mandatory_vehicles.rename(columns = {"EV_x": "EV"})

In [24]:
status_quo_vehicles.to_excel("FinalResults - status quo vehicles.xlsx")

In [25]:
mandatory_vehicles.to_excel("FinalResults - mandatory ZEDZ vehicles.xlsx")